In [3]:
from keras.models import Sequential
from keras.layers import Dense
one_way_model=Sequential()
one_way_model.add(Dense(32,activation='relu', input_shape=(50,)))
one_way_model.add(Dense(1, activation='sigmoid'))
one_way_model.summary()

Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                1632      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 1,665
Trainable params: 1,665
Non-trainable params: 0
_________________________________________________________________


In [4]:
from keras.models import Model
from keras.layers import Dense
from keras import Input
input_1=Input(shape=(50,))
dense_1=Dense(32, activation='relu')
x=dense_1(input_1)
dense_2=Dense(1,activation='sigmoid')
out_1=dense_2(x)
model_1=Model(input_1,out_1)
model_1.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1632      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 1,665
Trainable params: 1,665
Non-trainable params: 0
_________________________________________________________________


In [5]:
from keras.models import Model
from keras.layers import Dense
from keras import Input
input_2=Input(shape=(50,))
x=Dense(32, activation='relu')(input_2)
y=Dense(1,activation='sigmoid')(x)
model_2=Model(input_2,y)
model_2.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 50)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1632      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 33        
Total params: 1,665
Trainable params: 1,665
Non-trainable params: 0
_________________________________________________________________


In [5]:
# 다중 입력

from keras.models import Model
from keras.layers import Conv2D, GlobalMaxPooling2D, Embedding, LSTM, Dense, concatenate
from keras import Input

image_input=Input(shape=(128,128,3))
image_layer=Conv2D(64, (3,3), activation='relu')(image_input)
image_pool=GlobalMaxPooling2D()(image_layer)

maxlen=500 
text_input=Input(shape=(maxlen,))

# 1만개의 단어(정확히는 9999개의 단어)공간에서, 1개의 sample을 500개의 단어(500의 문장길이, 시간스탭)로 제한시킴(살펴볼거임). 
# 1개의 sample, (500,)벡터는 각 원소가 0부터 1만 이상의 값을 지닐 수 있음. 이를 1만으로 제한시킴.

# 그리고 결국, 120개의 feature로 뽑는 과정이라 생각하면 쉬움.

text_layer=Embedding(10000,120)(text_input) #  size가 120인 factor로 변환
text_lstm=LSTM(32)(text_layer)

age_input=Input(shape=(1,))
age_layer=Dense(5)(age_input)
concatenated=concatenate([image_pool, text_lstm, age_layer], axis=-1) # 마지막 축을 기준으로 더함.
price=Dense(1)(concatenated)
model=Model([image_input, text_input, age_input], price)
model.summary()

model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
model.fit([image, text, age], y_train, epochs=20, batch_size=128)

# 가정만 하고, 데이터는 없으므로 error가 뜬 상황.

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 500)          0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 126, 126, 64) 1792        input_5[0][0]                    
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 500, 120)     1200000     input_6[0][0]                    
____________________________________________________________________________________________

NameError: name 'image' is not defined

In [4]:
# 다중 출력

from keras.models import Model
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, Embedding, Dense
from keras import Input

maxlen=500
text_input=Input(shape=(maxlen,))
embedding_text=Embedding(10000,128)(text_input)
x=Conv1D(256,7,activation='relu')(embedding_text) # conv1D임에 명심. CNN + RNN 에 예로 진행했었음.

# 즉, CNN이 4D , image data로만 진행하는 것은 아니다.

x=MaxPooling1D(5)(x)
x=Conv1D(512,5,activation='relu')(x)
x=GlobalMaxPooling1D()(x) # time step으로 maxpooling 을 진행.
x=Dense(128,activation='relu')(x)
sex_out=Dense(1, activation='sigmoid', name='sex')(x)
age_out=Dense(1,name='age')(x)
job_out=Dense(10,activation='softmax', name='job')(x)
model=Model(text_input, [sex_out,age_out,job_out])
model.summary()

model.compile(optimizer='rmsprop', loss=['binary_crossentropy','mse','categorical_crossentropy'])
# loss를 어떻게 쓰는지에 대해 초점을 두고 보면 좋다.
# 이때, 전체 loss를 보면, 당연히 mse의 loss가 굉장히 클거임. 
# 따라서 weight를 주고, 전체 loss를 조절할 수 있음.

# loss = {'sex':'binary_crossentropy' ,'age':'mse' ,'job':'categorical_crossentropy'}
# loss_weight = {'sex':10,'age':0.25,'job':1.} 이런식으로 옵션가능.

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 500)          0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 500, 128)     1280000     input_4[0][0]                    
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, 494, 256)     229632      embedding_4[0][0]                
__________________________________________________________________________________________________
max_pooling1d_4 (MaxPooling1D)  (None, 98, 256)      0           conv1d_7[0][0]                   
____________________________________________________________________________________________